# Demo of iRF classification

* The following is a demo of the scikit learn iRF with FP-Growth code

## Typical Setup

### Import the required dependencies

* In particular `irf_utils` and `irf_jupyter_utils`

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import numpy as np
from functools import reduce

# Needed for the scikit-learn wrapper function
from irf import irf_utils
from irf.ensemble import RandomForestClassifier
from math import ceil

# Import our custom utilities
from imp import reload
from irf import irf_jupyter_utils
reload(irf_jupyter_utils)

import pandas as pd

/Users/ericxia/anaconda/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
/Users/ericxia/anaconda/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


ImportError: cannot import name 'RandomForestClassifierWithWeights'

## Step 1: Fit the Initial Random Forest

* Just fit every feature with equal weights per the usual random forest code e.g. DecisionForestClassifier in scikit-learn

In [ ]:
X_train, X_test, y_train, y_test, rf = irf_jupyter_utils.generate_rf_example(n_estimators=20, 
                                                                             feature_weight=None)

## Check out the data

In [ ]:
print("Training feature dimensions", X_train.shape, sep = ":\n")
print("\n")
print("Training outcome dimensions", y_train.shape, sep = ":\n")
print("\n")
print("Test feature dimensions", X_test.shape, sep = ":\n")
print("\n")
print("Test outcome dimensions", y_test.shape, sep = ":\n")
print("\n")
print("first 2 rows of the training set features", X_train[:2], sep = ":\n")
print("\n")
print("first 2 rows of the training set outcomes", y_train[:2], sep = ":\n")

## Step 2: Get all Random Forest and Decision Tree Data

* Extract in a single dictionary the random forest data and for all of it's decision trees
* This is as required for FP-Growth purposes

In [ ]:
all_rf_tree_data = irf_utils.get_rf_tree_data(
    rf=rf, X_train=X_train, X_test=X_test, y_test=y_test)

# Step 3: Plot some Data

### List Ranked Feature Importances

In [ ]:
# Print the feature ranking
print("Feature ranking:")

feature_importances_rank_idx = all_rf_tree_data['feature_importances_rank_idx']
feature_importances = all_rf_tree_data['feature_importances']

for f in range(X_train.shape[1]):
    print("%d. feature %d (%f)" % (f + 1
                                   , feature_importances_rank_idx[f]
                                   , feature_importances[feature_importances_rank_idx[f]]))

### Plot Ranked Feature Importances


In [ ]:
# Plot the feature importances of the forest
feature_importances_std = all_rf_tree_data['feature_importances_std']

plt.figure()
plt.title("Feature importances")
plt.bar(range(X_train.shape[1])
        , feature_importances[feature_importances_rank_idx]
        , color="r"
        , yerr = feature_importances_std[feature_importances_rank_idx], align="center")
plt.xticks(range(X_train.shape[1]), feature_importances_rank_idx)
plt.xlim([-1, X_train.shape[1]])
plt.show()

## Decision Tree 0 (First) - Get output

### Check the output against the decision tree graph

In [ ]:
# Now plot the trees individually
irf_jupyter_utils.draw_tree(decision_tree = all_rf_tree_data['rf_obj'].estimators_[0])

# Run the iRF function

We will run the iRF FP-Growth with the following **parameters**

#### Data:
* breast cancer binary classification data
* **random state (for reproducibility):** 2018

#### Weighted RFs
* **K:** 5 iterations
* **number of trees:** 20

#### Bootstrap RFs
* **proportion of bootstrap samples:** 20%
* **B:** 30 bootstrap samples
* **number of trees (bootstrap RFs):** 5 iterations

#### FP-Growth (on the bootstrap RFs)
* **min_support:** 0.05 (interaction must occur 5% of the time to be returned by FP-Growth)
* **min_confidence:** 0.8 (irrelevant currently for iRF purposes but maybe useful for others)
* **bootstrap_num:** 20 (for each bootstrap sample, saves top 10 interactions and then computes stability)

# Running the iRF is easy - single function call

* All of the bootstrap, RIT complexity is covered through the key parameters passed through
in the main algorithm (as listed above)
* This function call returns the following data:
    1. all RF weights
    2. all the K RFs that are iterated over
    3. all of the B bootstrap RFs that are run
    4. all the B*M RITs that are run on the bootstrap RFs
    5. the stability score
    
### This is a lot of data returned!

Will be useful when we build the **interface** later

### Let's run it!

In [ ]:
all_rf_weights, all_K_iter_rf_data, \
all_rf_bootstrap_output, all_rit_bootstrap_output, \
stability_score = irf_utils.run_iRF_FPGrowth(X_train=X_train,
                                    X_test=X_test,
                                    y_train=y_train,
                                    y_test=y_test,
                                    K=5,
                                    rf=RandomForestClassifier(n_estimators=40),
                                    B=30,
                                    random_state_classifier=2018,
                                    propn_n_samples=.2,
                                    bin_class_type=1,
                                    min_support=0.05,
                                    min_confidence=0.8,
                                    bootstrap_num=20)

In [ ]:
stability_score

# Examine the stability scores

In [ ]:
irf_jupyter_utils._get_histogram(stability_score, sort = True)

## Comparing Stability Scores between RIT iRF and FP-Growth iRF

In [ ]:
_, _, _, _, rit_stability_score = irf_utils.run_iRF(X_train=X_train,
                                    X_test=X_test,
                                    y_train=y_train,
                                    y_test=y_test,
                                    K=5,
                                    rf=RandomForestClassifier(n_estimators=40),
                                    B=30,
                                    random_state_classifier=2018,
                                    propn_n_samples=.2,
                                    bin_class_type=1,
                                    M=20,
                                    max_depth=5,
                                    noisy_split=False,
                                    num_splits=2,
                                    signed=False)

In [ ]:
def unsigned_to_output(path):
    temp = path.split("_")
    features = [int(elem) for elem in temp]
    features.sort()
    features = [str(elem) for elem in features]
    output = "_".join(features)
    return output

In [ ]:
from collections import defaultdict
conversion = defaultdict(list)
rit_interactions = []
interactions = []

for inter in stability_score.keys():
    conversion[unsigned_to_output(inter)].append(inter)
    rit_interactions.append(unsigned_to_output(inter))
    
for inter in rit_stability_score.keys():
    conversion[unsigned_to_output(inter)].append(inter)
    interactions.append(unsigned_to_output(inter))

In [ ]:
rit_interactions = set(rit_interactions)
interactions = set(interactions)

print("The number of RIT interactions are:")
print(len(interactions))

print("The number of FP-Growth interactions are:")
print(len(rit_interactions))

print("The number of interactions in common are:")
print(len(rit_interactions.intersection(interactions)))

It seems that both RIT and FP-Growth are returning many interactions in common, and most of the FP-Growth interactions are ones also found by RIT. However RIT iRF is returning more interactions as compared to FP-Growth, but this can easily be adjusted by changing the min_support parameter.

In [ ]:
rit_sta = []
fp_sta = []
feature_paths = []

data_x = rit_stability_score.keys()
data_x = sorted(rit_stability_score, key=rit_stability_score.get,
                        reverse=True)

for inter in data_x:
    index = unsigned_to_output(inter)
    rit_sta.append(rit_stability_score[index])
    all_inter = conversion[index]
    if len(all_inter) == 1:
        fp_sta.append(0)
    else:
        index = all_inter.index(inter)
        temp = all_inter[:index] + all_inter[index+1:]
        fp_sta.append(stability_score[temp[0]])
    feature_paths.append(inter)
    
for inter in stability_score.keys():
    index = unsigned_to_output(inter)
    all_inter = conversion[index]
    if len(all_inter) == 1:
        rit_sta.append(0)
        fp_sta.append(stability_score[all_inter[0]])
        feature_paths.append(index)

In [ ]:
plt.clf()
plt.figure(figsize=(15, 8))

# code copied from stack overflow
def subcategorybar(X, vals, width=0.8):
    n = len(vals)
    _X = np.arange(len(X))
    plt.bar(_X - width/2. + 0/float(n)*width, vals[0], 
            width=width/float(n), align="edge", label="RIT") 
    plt.bar(_X - width/2. + 1/float(n)*width, vals[1], 
            width=width/float(n), align="edge", label="FP-Growth")   
    #plt.xticks(_X, X)
    
subcategorybar(feature_paths, [rit_sta, fp_sta])
plt.legend()

plt.ylabel("Stability Score")
plt.title("Stability Scores of Features Paths, RIT iRF vs FP-Growth iRF")

plt.show()

For every interaction that iRF with RIT returns, its stability score is plotted in blue, and right next to it in orange is the corresponding FP-Growth stability score for the feature. A few things to point out, it seems that FP-Growth has a tendency to have higher stability scores for the interactions that the two find in common. In addition, some of the interactions that FP-Growth find but RIT doesn't (or RIT assigns low stability scores for) FP-Growth gives fairly high stability scores.

## Speed Comparison

In [ ]:
import time

start = time.time()
_, _, _, _, _ = irf_utils.run_iRF(X_train=X_train,
                                    X_test=X_test,
                                    y_train=y_train,
                                    y_test=y_test,
                                    K=5,
                                    rf=RandomForestClassifier(n_estimators=40),
                                    B=30,
                                    random_state_classifier=2018,
                                    propn_n_samples=.2,
                                    bin_class_type=1,
                                    M=20,
                                    max_depth=5,
                                    noisy_split=False,
                                    num_splits=2,
                                    signed=False)
end = time.time()
RIT_time = end - start

start = time.time()
_, _, _, _, _ = irf_utils.run_iRF_FPGrowth(X_train=X_train,
                                    X_test=X_test,
                                    y_train=y_train,
                                    y_test=y_test,
                                    K=5,
                                    rf=RandomForestClassifier(n_estimators=40),
                                    B=30,
                                    random_state_classifier=2018,
                                    propn_n_samples=.2,
                                    bin_class_type=1,
                                    min_support=0.05,
                                    min_confidence=0.8,
                                    bootstrap_num=20)
end = time.time()
FPGrowth_time = end - start

In [ ]:
time = (RIT_time, FPGrowth_time)
objects = ("RIT", "FP-Growth")

plt.bar([0, 1], time, align='center', alpha=0.5)
plt.xticks([0,1], objects)
plt.ylabel('Time (s)')
plt.title('iRF Time to Run')

plt.show()

FP-Growth is about 5 seconds slower (25% slower), but most of this is likely due to overheard for intializing Spark. Would need to run on datasets increasing in size to see overall performance trend.

Potential Enhancements: Instead of getting all the interactions that have greater than some specified support, there are algorithms (TFP) based on FP-Growth that are able to return the top $k$ closed interaction sets. This would likely provide substantial speedup and would not require the user to pre-specify the required minimum support. However the issue is that there is no publicly available implementation of such algorithms.